# Proxecto 3: Aprendizaxe semisupervisado

## Carga e preprocesado dos datos

In [5]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds


train = tfds.load('stl10', split = 'train', as_supervised = True)
test = tfds.load('stl10', split = 'test', as_supervised = True)
unlabeled = tfds.load('stl10', split = 'unlabelled', as_supervised = True)

train = train.map(lambda x, y: (tf.image.resize(x, (32, 32)), y)) 
test = test.map(lambda x, y: (tf.image.resize(x, (32, 32)), y))
unlabeled = unlabeled.map(lambda x, y: (tf.image.resize(x, (32, 32)), y))

x_train = train.map(lambda x, y: x/255)
y_train = train.map(lambda x, y: y)

x_test = test.map(lambda x, y: x/255)
y_test = test.map(lambda x, y: y)

unlabeled = unlabeled.map(lambda x, y: x/255)

x_train = tfds.as_numpy(x_train)
y_train = tfds.as_numpy(y_train)

x_test = tfds.as_numpy(x_test)
y_test = tfds.as_numpy(y_test)

unlabeled = tfds.as_numpy(unlabeled)

x_train = np.vstack(map(lambda x: np.expand_dims(x, 0), x_train))
y_train = np.vstack(y_train)

x_test = np.vstack(map(lambda x: np.expand_dims(x, 0), x_test))
y_test = np.vstack(y_test)

unlabeled = np.vstack(map(lambda x: np.expand_dims(x, 0), unlabeled))

C:\Users\uxiom\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\uxiom\AppData\Local\Temp\ipykernel_19112\1290518901.py:30: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  x_train = np.vstack(map(lambda x: np.expand_dims(x, 0), x_train))
C:\Users\uxiom\AppData\Local\Temp\ipykernel_19112\1290518901.py:31: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  y_train = np.vstack(y_

In [12]:
# Comprobacion do tamaño dos datos
print("x_train:", len(x_train), "x_test:", len(x_test), "x_unlabeled:", len(unlabeled))

x_train: 5000 x_test: 8000 x_unlabeled: 100000


One-hot encoding

In [13]:
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

## Creación do modelo inicial

In [160]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
for i in range(4):
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [142]:
model.summary()

Model: "sequential_74"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_155 (Conv2D)         (None, 30, 30, 32)        896       
                                                                 
 conv2d_156 (Conv2D)         (None, 28, 28, 32)        9248      
                                                                 
 conv2d_157 (Conv2D)         (None, 26, 26, 32)        9248      
                                                                 
 conv2d_158 (Conv2D)         (None, 24, 24, 32)        9248      
                                                                 
 conv2d_159 (Conv2D)         (None, 22, 22, 32)        9248      
                                                                 
 flatten_23 (Flatten)        (None, 15488)             0         
                                                                 
 dense_47 (Dense)            (None, 10)              

In [162]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [144]:
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
157/157 [==============================] - 20s 117ms/step - loss: 1.8750 - categorical_accuracy: 0.2960
Epoch 2/10
157/157 [==============================] - 18s 113ms/step - loss: 1.5604 - categorical_accuracy: 0.4162
Epoch 3/10
157/157 [==============================] - 18s 113ms/step - loss: 1.3973 - categorical_accuracy: 0.4800
Epoch 4/10
157/157 [==============================] - 18s 112ms/step - loss: 1.2600 - categorical_accuracy: 0.5350
Epoch 5/10
157/157 [==============================] - 18s 113ms/step - loss: 1.1156 - categorical_accuracy: 0.5908
Epoch 6/10
157/157 [==============================] - 18s 114ms/step - loss: 0.9605 - categorical_accuracy: 0.6538
Epoch 7/10
157/157 [==============================] - 18s 114ms/step - loss: 0.7545 - categorical_accuracy: 0.7318
Epoch 8/10
157/157 [==============================] - 18s 117ms/step - loss: 0.5771 - categorical_accuracy: 0.7950
Epoch 9/10
157/157 [==============================] - 18s 114ms/step - loss: 0.3

In [145]:
model.evaluate(x_train, y_train)
model.evaluate(x_test, y_test)

250/250 [==============================] - 5s 19ms/step - loss: 3.2164 - categorical_accuracy: 0.4305


[3.2164132595062256, 0.43050000071525574]

1. ¿Qué red has escogido? ¿Cómo la has entrenado?

    A rede debía ser completamente convolucional, polo que o modelo leva 5 capas convolucionais, todas con 32 filtros e un kernel de tamaño (3, 3). Ademáis leva unha capa Flatten() previa á capa de saída Dense, con 10 neuronas para predecir as 10 clases posíbeis, con activación 'softmax'. A rede foi entrenada durante 10 épocas, con un batch_size de 32 e un optimizador Adam cunha taxa de aprendizaxe de 0.001. A función de pérdida foi CategoricalCrossentropy, e a métrica Accuracy.
 
2. ¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?

    A precisión do modelo en entrenamento é do 96.20%, e en test do 43.05%.

3. ¿Qué conclusiones sacas de los resultados del punto anterior?

    A diferenza entre a precisión de entrenamento e a de test é moi grande, polo que o modelo padece overfitting e sobreadestra. Isto pode ser debido a que o modelo non ten conxunto de validación nin capas de regularización que frenen este sobreadestramento, polo que o modelo aprende moi ben os datos de entrenamento, pero non xeneraliza ben para os datos de test.

## Auto aprendizaxe

In [158]:
def self_training_v2(model_func, x_train, y_train, unlabeled, thresh=0.8, train_epochs=3):
    train_data = x_train.copy()
    train_labels = y_train.copy()
    labeled_weight = 1 # Asigna un peso maior aos datos etiquetados (certeza)
    train_weight = np.array([labeled_weight]*len(train_labels))
    for i in range(train_epochs):
        lista_unlabeled = []
        model = model_func
        model.fit(train_data, train_labels, sample_weight=train_weight) # Usa sample_weight para asignar o peso
        y_pred = model.predict(unlabeled)
        y_class, y_value = y_pred.argmax(axis = 1), y_pred.max(axis = 1) # Escolle a clase segundo a confianza máis alta e devolve tamen esa confianza
        # Recorre cada valor da tupla (unlabeled_data, y_class, y_value)
        for i in range(len(unlabeled)):
            if y_value[i] > thresh:
                # Asigna como peso dos datos sen etiquetar a súa confianza 
                lista_unlabeled.append(i)
        train_data = np.concatenate((train_data, unlabeled[lista_unlabeled]), axis = 0)
        train_labels = np.concatenate((train_labels, tf.keras.utils.to_categorical(y_class[lista_unlabeled], 10)), axis=0)
        train_weight = np.concatenate([train_weight, y_value[lista_unlabeled]], axis=0)
            
    return model

In [ ]:
model_st = self_training_v2(model, x_train, y_train, unlabeled, thresh=0.8, train_epochs=10)

In [ ]:
model_st.evaluate(x_train, y_train)
model_st.evaluate(x_test, y_test)

1. ¿Qué parámetros has definido para el entrenamiento?

    O modelo inicial é o mesmo ca o do apartado anterior, polo que os parámetros iniciais son os mesmos. Ademáis, ao incluir os datos non clasificados, especificamos o parámetro 'labeled_weight' para que o modelo dea máis importancia aos datos clasificados que aos non clasificados. Durante o adestramento, en 'train_weight' vanse ir gardando a confianza do modelo para cada dato non clasificado, para ponderalo con respecto aos etiquetados. En 'lista_unlabeled' almacenamos os índices dos datos non etiquetados para os que o modelo ten unha certeza coa clasificación superior ao umbral (0.8 neste caso), para engadilos ao conxunto de datos de adedstramento.

2. ¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?

    La precisión del modelo en entrenamiento es del 41.20%, y en test del 33.05%.

3. ¿Se mejoran los resultados obtenidos en el ejercicio anterior?

    Non, empeoran. Evítase o sobreaxuste pero non melloran os resultados.

4. ¿Qué conclusiones sacas de los resultados del punto anterior?

## Aprendizaxe semisupervisado de tipo autoencoder 

In [51]:
class Autoencoder:

    def __init__(self, input_shape):

        self.encoder = tf.keras.Sequential()
        self.encoder.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        for i in range(4):
            self.encoder.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
        self.encoder.add(tf.keras.layers.MaxPooling2D((2, 2)))

        self.decoder = tf.keras.Sequential()
        self.decoder.add(tf.keras.layers.Conv2DTranspose(32, (3, 3), activation='relu'))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(16, (5, 5), activation='relu'))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(16, (5, 5), activation='relu'))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(8, (6, 6), activation='relu'))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(3, (7, 7), activation='relu'))

        self.autoencoder = tf.keras.Sequential([self.encoder, self.decoder])

        self.autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

    def fit(self, X, y, epochs=5, batch_size=32):
        print(self.autoencoder.summary())
        self.autoencoder.fit(X, y, epochs=epochs, batch_size=batch_size)

    def get_encoded_data(self, X, batch_size = 32):
        return self.encoder.predict(X, batch_size = batch_size)
    
    def _del_(self):
        tf.keras.backend.clear_session()

In [58]:
class Clasificador:
    def __init__(self):
        self.classifier = tf.keras.Sequential()
        self.classifier.add(tf.keras.layers.Flatten())
        self.classifier.add(tf.keras.layers.Dense(25, activation='relu'))
        self.classifier.add(tf.keras.layers.Dense(15, activation='relu'))
        self.classifier.add(tf.keras.layers.Dense(10, activation='softmax'))

        self.classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy()])

    def fit(self, X, y, epochs=5, batch_size=32):
        self.classifier.fit(X, y, epochs=epochs, batch_size=batch_size)
    
    def predict(self, X, batch_size=32):
        return self.classifier.predict(X, batch_size=batch_size)
    
    def predict_proba(self, X):
        return self.clasificador.predict_proba(X)
    
    def score(self, X, y):
        return self.classifier.evaluate(X, y)
    
    def _del_(self):
        tf.keras.backend.clear_session()

In [3]:
def semisupervised_training(autoencoder, clasificador, x_train, y_train, unlabeled):
    
    autoencoder_data = np.concatenate((x_train, unlabeled), axis=0)
    autoencoder.fit(autoencoder_data, autoencoder_data)

    clasificador_data = autoencoder.get_encoded_data(x_train)
    clasificador.fit(clasificador_data, y_train)

In [59]:
autoencoder = Autoencoder((32, 32, 3))
clasificador = Clasificador()
semisupervised_training(autoencoder, clasificador, x_train, y_train, unlabeled)

Model: "sequential_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_62 (Sequential)  (None, 11, 11, 32)        37888     
                                                                 
 sequential_63 (Sequential)  (None, 32, 32, 3)         34275     
                                                                 
Total params: 72,163
Trainable params: 72,163
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
3282/3282 [==============================] - 606s 184ms/step - loss: 0.0243 - mse: 0.0243
Epoch 2/5
3282/3282 [==============================] - 586s 179ms/step - loss: 0.0176 - mse: 0.0176
Epoch 3/5
3282/3282 [==============================] - 1265s 386ms/step - loss: 0.0164 - mse: 0.0164
Epoch 4/5
3282/3282 [==============================] - 722s 220ms/step - loss: 0.0146 - mse: 0.0146
Epoch 5/5
157/157 [====================

Avaliación:

In [60]:
pred_data = autoencoder.get_encoded_data(x_test)
clasificador.score(pred_data, y_test)

250/250 [==============================] - 1s 3ms/step - loss: 2.0111 - categorical_accuracy: 0.1924


[2.0111160278320312, 0.19237500429153442]

1. ¿Cuál es la arquitectura del modelo? ¿Y sus hiperparámetros (epochs, batch size, optimizador, ...)?


    A arquitetura do encoder é igual á do modelo inicial, mentres que para o decoder utilizamos 5 capas Conv2DTranspose, con filtros descendentes e un kernel que aumenta de tamaño. Foi adestrado con 5 epochs e tamaño de lote 32. 
    En canto ao clasificador, e un clasificador moi sinxelo de 3 capas Dense, rematando nunha capa de saída de 10 neuronas, con activación 'softmax'. Foi adestrado con 10 epochs e tamaño de lote 32.

2. ¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?


    En test, a precisión do modelo é do 19.24%.

3. ¿Se mejoran los resultados obtenidos en los puntos anteriores?


    Non, de feito empeóranse bastante. Entendemos que pode ser por utilizar uns epochs reducidos (para que se executara nun tempo razoábel) e por ter un clasificador sinxelo, que non é capaz de aprender ben os datos.

4. ¿Qué conclusiones sacas de este apartado?


    É difícil sacar conclusións ao ter tan malos resultados, xa que aínda que supoñemos que parte da culpa é do clasificador, non sabemos se o autoencoder está ben implementado ou non.

# Aprendizaxe de tipo autoencoder en 1 paso

In [142]:
class MiClasificadorSemisupervisado:

    def __init__(self, input_shape):

        self.encoder = tf.keras.Sequential()
        self.encoder.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
        for i in range(4):
            self.encoder.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
        self.encoder.add(tf.keras.layers.MaxPooling2D((2, 2)))

        self.decoder = tf.keras.Sequential()
        self.decoder.add(tf.keras.layers.Conv2DTranspose(32, (3, 3), activation='relu'))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(16, (5, 5), activation='relu'))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(16, (5, 5), activation='relu'))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(8, (6, 6), activation='relu'))
        self.decoder.add(tf.keras.layers.Conv2DTranspose(3, (7, 7), activation='relu'))

        self.clasificador = tf.keras.Sequential()
        self.clasificador.add(tf.keras.layers.Flatten())
        self.clasificador.add(tf.keras.layers.Dense(15, activation='softmax'))
        self.clasificador.add(tf.keras.layers.Dense(10, activation='softmax'))

        self.autoencoder = tf.keras.Sequential([self.encoder, self.decoder])
        self.clasif = self.clasificador

        inputs = tf.keras.Input(shape=input_shape)
        enc_out = self.autoencoder(inputs)
        clasif_out = self.clasif(enc_out)
        self.model = tf.keras.Model(inputs=inputs, outputs=[enc_out, clasif_out])

        self.model.compile(optimizer='adam', loss = ['mse', 'categorical_crossentropy'], metrics = ['mse', 'categorical_accuracy'])

    def fit(self, X, y, unlabeled = None, epochs=5, batch_size=32):
        
        if unlabeled is None:
            X_train = X
            y_train = y
        else:
            X_train = np.concatenate((X, unlabeled))
            y_train = np.concatenate((y, np.zeros((unlabeled.shape[0], 10))))
            
        y_train_autoencoder = X_train
        y_train_clasificador = y_train
        
        self.model.fit(X_train, [y_train_autoencoder, y_train_clasificador], epochs=epochs, batch_size=batch_size)

        
    def predict(self, X):
        return np.argmax(self.model.predict(X)[1])
    
    def predict_proba(self, X):
        return self.model.predict(X)[1]
    
    def score(self, X, y):
        self.model.evaluate(X, [X, y])

    def __del__(self):
        tf.keras.backend.clear_session() 

In [137]:
autoencoder_clasificador = MiClasificadorSemisupervisado((32, 32, 3))
autoencoder_clasificador.fit(x_train, y_train, unlabeled)

Epoch 1/5
3282/3282 [==============================] - 643s 195ms/step - loss: 0.1356 - sequential_16_loss: 0.0254 - sequential_15_loss: 0.1102 - sequential_16_mse: 0.0254 - sequential_16_categorical_accuracy: 0.4783 - sequential_15_mse: 0.0141 - sequential_15_categorical_accuracy: 0.0048
Epoch 2/5
3282/3282 [==============================] - 601s 183ms/step - loss: 0.1282 - sequential_16_loss: 0.0180 - sequential_15_loss: 0.1102 - sequential_16_mse: 0.0180 - sequential_16_categorical_accuracy: 0.5141 - sequential_15_mse: 0.0140 - sequential_15_categorical_accuracy: 0.0048
Epoch 3/5
3282/3282 [==============================] - 578s 176ms/step - loss: 0.1253 - sequential_16_loss: 0.0151 - sequential_15_loss: 0.1101 - sequential_16_mse: 0.0151 - sequential_16_categorical_accuracy: 0.5979 - sequential_15_mse: 0.0140 - sequential_15_categorical_accuracy: 0.0048
Epoch 4/5
3282/3282 [==============================] - 569s 173ms/step - loss: 0.1238 - sequential_16_loss: 0.0137 - sequential_15

In [144]:
autoencoder_clasificador.score(x_test, y_test)

250/250 [==============================] - 17s 67ms/step - loss: 2.3242 - sequential_16_loss: 0.0131 - sequential_15_loss: 2.3111 - sequential_16_mse: 0.0131 - sequential_16_categorical_accuracy: 0.6287 - sequential_15_mse: 0.0902 - sequential_15_categorical_accuracy: 0.1000


1. ¿Cuál es la arquitectura del modelo? ¿Y sus hiperparámetros (epochs, batch size, optimizador, ...)?

    A arquitetura do autoencoder é igual ca no anterior apartado. En canto ao clasificador, retiramos a primeira capa con respecto ao apartado anterior, quedando con dúas de 15 e 10 neuronas, respectivamente. Foi adestrado con 5 epochs e tamaño de lote 32.

2. ¿Cuál es la precisión del modelo en entrenamiento? ¿Y en test?

    Ao implementar varias métricas, xa que estamos adestrando nun paso un autoencoder e un clasificador, o resultado non é moi visual, pero a precisión do autoencoder correspóndese con sequential_16_categorical_accuracy e ten un valor de 62.87% en test. A precisión do clasificador é sequential_15_categorical_accuracy, e resultou dun 10% en test.

3. ¿Se mejoran los resultados obtenidos en los puntos anteriores?

    O accuracy final do clasificador empeora, pero ao observar por separado o do autoencoder, vemos que é bastante superior ao do clasificador. p

4. ¿Qué conclusiones sacas de este apartado?

    Confirmamos a nosa hipótese do apartado anterior de que a baixa precisión final debíase ao clasificador. O autoencoder está ben implementado, xa que aprende ben a representación latente dos datos.